In [2]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt 
import matplotlib as mpl
import seaborn as sns 
import geopandas as gpd
import pandas as pd 
import numpy as np
import requests
import shapely
import pickle
import progressbar
import geopy.distance
from shapely.geometry import Point, Polygon, LineString 
from shapely.ops import nearest_points
from shapely.ops import transform
from shapely import geometry, ops
from functools import partial
from itertools import combinations
import geopandas as gpd
import pyproj
import os
import matplotlib.cm as cm
import matplotlib.colors as colors
from scipy.sparse import lil_matrix
import itertools
import warnings
warnings.simplefilter('ignore')
import random
from random import choice, randint
import timeit
import time
import threading
from multiprocessing import Pool
from joblib import Parallel, delayed
#os.chdir('C:\\Users\\piene\\Dropbox\\Python')
#os.getcwd()

C:\Users\alexa\anaconda3\envs\ox\lib\site-packages\pyproj\__init__.py:91: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_LIB or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))


In [3]:
path = "Data/B10000-CS2.gpickle"
B = nx.read_gpickle(path)
df = pd.read_pickle("Data/flows.csv")
df2 = pd.read_pickle("Data/flows_unsimp.csv")
path = "Data/H.gpickle"
#nx.write_gpickle(H, path)
H = nx.read_gpickle(path)
H = H.to_undirected()
CS = pd.read_csv("Data/AllCurrentCS.csv", encoding="Cp1252")
CS = CS.drop('Unnamed: 0', axis=1)
G = nx.read_gpickle("Data/unsimp.gpickle")

with open('Data\AllCSnodes.pickle', 'rb') as handle:
    AllCSnodes = pickle.load(handle)
with open('Data\lengths.pickle', 'rb') as handle:
    lengths_dict = pickle.load(handle)

In [4]:
def calculate_eff_lengths(edge):
    
    """
    Calculates the effective length of an edge by using length weights and road gradient
    """

    startnode = edge[0]
    endnode = edge[1]
    const = 0.372
    grade_intervals = [-0.09, -0.07, -0.05, -0.03, -0.01, 0.01, 0.03, 0.05, 0.07, 0.09, 0.11]
    increased_consumption = [-0.332, -0.217, -0.148, -0.121, -0.073, 0.085, 0.152, 0.203, 0.306, 0.358, 0.552]
    
    iterator = len(B.edges[edge]['length_weight'])
    
    #If the direction of the edge is the default fromnode -> tonode, we set a multiplier to 1, meaning that the edge gradients will be default:
    if B.nodes[startnode]['elevation'] < B.nodes[endnode]['elevation']: 
        direction = 1
    #If direction is reversed, meaning that the vehicle is returning to the origin node, we reverse the gradients
    else:
        direction = -1
    eff_length_trueway = 0
        
    for i in range(iterator):
        edge_grad = B.edges[edge]['grade'][i]*(direction)
        length_weight = B.edges[edge]['length_weight'][i]

        if edge_grad < grade_intervals[0]: #less than -9%
            diff = const + increased_consumption[0]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[0] < edge_grad <= grade_intervals[1]: #-9% to -7%
            diff = const + increased_consumption[1]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[1] < edge_grad <= grade_intervals[2]: #-7% to -5%
            diff = const + increased_consumption[2]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[2] < edge_grad <= grade_intervals[3]: #-5% to -3%
            diff = const + increased_consumption[3]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[3] <= edge_grad <= grade_intervals[4]: #-3% to -1%
            diff = const + increased_consumption[4]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[5] <= edge_grad <= grade_intervals[6]: #1% to #3%
            diff = const + increased_consumption[5]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[6] < edge_grad <= grade_intervals[7]: #3% to 5%
            diff = const + increased_consumption[6]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[7] < edge_grad <= grade_intervals[8]: #5% to 7%
            diff = const + increased_consumption[7]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[8] < edge_grad <= grade_intervals[9]: #7% to 9%
            diff = const + increased_consumption[8]
            eff_length_trueway += length_weight*(diff/const)

        elif grade_intervals[9] < edge_grad <= grade_intervals[10]: #9% to 11%
            diff = const + increased_consumption[9]
            eff_length_trueway += length_weight*(diff/const)

        elif edge_grad > grade_intervals[10]: #more then 11%
            diff = const + increased_consumption[0]
            eff_length_trueway += length_weight*(diff/const)


        else: #If grade is between -1% and 1%
            eff_length_trueway += length_weight
            diff = 0
     
    return eff_length_trueway
        

In [5]:
maxdegreesimp = 0
degreessimp = nx.degree(B)
alldegreessimp = []
for i in degreessimp:
    if i[1] > maxdegreesimp:
        maxdegreesimp = i[1]
    alldegreessimp.append(i[1])
    
pathlengthssimp = []
maxpathlengthsimp = 0
for i in range(len(df)):
    temp = len(df.iloc[i]['path'])
    pathlengthssimp.append(temp)
    if temp > maxpathlengthsimp:
        maxpathlengthsimp=temp
avgpathlengthsimp = np.mean(pathlengthssimp)

In [6]:
#Calculate network statistics
nodessimp = B.number_of_nodes()
edgessimp = B.number_of_edges()
densitysimp = nx.density(B)
avgdegreesimp = np.mean(alldegreessimp)
maxdegreesimp
allitems = [['Nodes',nodessimp],['Edges',edgessimp],['Density',densitysimp],['Average Degree', avgdegreesimp], ['Max degree', maxdegreesimp],['Average Path Length', avgpathlengthsimp], ["Max Path Length",maxpathlengthsimp]]
    



In [7]:
data = pd.DataFrame(data=allitems, columns=["Statistics","Value"])


In [8]:
maxdegrees = 0
degrees = nx.degree(H)
alldegrees = []
for i in degrees:
    if i[1] > maxdegrees:
        maxdegrees = i[1]
    alldegrees.append(i[1])
    
pathlengths = []
maxpathlength = 0
for i in range(len(df2)):
    temp = len(df2.iloc[i]['path'])
    pathlengths.append(temp)
    if temp > maxpathlength:
        maxpathlength=temp
avgpathlength = np.mean(pathlengths)

In [9]:
nodes = H.number_of_nodes()
edges = H.number_of_edges()
density = nx.density(H)
avgdegree = np.mean(alldegrees)

lengths_simp = []
lengths = []

for i in H.edges:
    lengths.append(H.edges[i]['length'])
    
for i in B.edges:
    lengths_simp.append(B.edges[i]['length'])
    
avglength = np.mean(lengths)
avglength_simp = np.mean(lengths_simp)
print(avglength, avglength_simp)

#allitems = [['Nodes',nodes],['Edges',edges],['Density',density],['Average Degree', avgdegree], ['Max degree', maxdegrees],['Average Path Length', avgpathlength], ["Max Path Length",maxpathlength]]
allitems = [['Nodes',nodessimp, nodes],['Edges',edgessimp,edges],['Density',densitysimp,density],['Average Degree', avgdegreesimp,avgdegree], ['Max degree', maxdegreesimp,maxdegrees],['Average Path Length', avgpathlengthsimp,avgpathlength], ["Max Path Length",maxpathlengthsimp,maxpathlength],["Avg Dist between each node",avglength_simp, avglength]]   


180.49781259963456 8314.482955396483


In [10]:
data = pd.DataFrame(data=allitems, columns=["Statistics","Simp","Unsimp"])
data


,Statistics,Simp,Unsimp
0,Nodes,1583.000000,205531.000000
1,Edges,1918.000000,217581.000000
2,Density,0.001532,0.000010
3,Average Degree,2.423247,2.117257
4,Max degree,7.000000,5.000000
5,Average Path Length,91.275610,3119.946341
6,Max Path Length,297.000000,9697.000000
7,Avg Dist between each node,8314.482955,180.497813


In [10]:
def Objectivefunction(FacilitiesList, paths, flows, lengths_dict, G_dict, Totalrange):
    startime = time.time()
    CoveredObjectiveValue = 0
    facilities = tuple(FacilitiesList)
    
    for j in range(len(paths)):
        path = tuple(paths[j])
        NodeInFacilityListExistInPath = False
        if (lengths_dict[path[0],path[-1]]+lengths_dict[path[-1],path[0]]) <=Totalrange/2: # If the round trip length is shorter than half the EV range, add the flow immediately 
            CoveredObjectiveValue += flows[j]
        
        else:
            for node in path:
                if node in facilities:
                    NodeInFacilityListExistInPath = True
                    break
                    
                    
        if NodeInFacilityListExistInPath == True: #Only evaluate if at least one facility is on the path
            
            # Take first from Origin to Destination:
            if path[0] in facilities:
                rem_range = Totalrange
            else:
                rem_range = Totalrange/2
                
            indicator_canRefuel=0
                
            #Start looping through the path going first from origin to the destination:
            revpath = path[::-1][1:]
            fullpath = path+revpath                    
            
            for i in range(len(fullpath)+1):                
                a = fullpath[i]
                b = fullpath[i+1]                                                                                     
                rem_range -= G_dict[(a,b)]

                if rem_range <0:
                    break
                elif b == path[-1] & path[-1] in facilities:
                    indicator_canRefuel+=1
                    break
                elif b == path[0]:
                    indicator_canRefuel+=1
                    break
                elif b in facilities:
                    rem_range = Totalrange
            
            if indicator_canRefuel==1:
                
                
                # Now consider from Destination to Origin:
                dest_orig_path = path[::-1]
                fullpath = dest_orig_path + path[1:]

                if fullpath[0] in facilities:
                    rem_range = Totalrange
                else:
                    rem_range = Totalrange/2

                for i in range(len(fullpath)+1):            
                    a = fullpath[i]
                    b = fullpath[i+1]                                                                                     
                    rem_range -= G_dict[(a,b)]

                    if rem_range <0:
                        break
                    elif b == dest_orig_path[-1] & dest_orig_path[-1] in facilities:
                        indicator_canRefuel+=1
                        break
                    elif b == dest_orig_path[0]:
                        indicator_canRefuel+=1
                        break
                    elif b in facilities:
                        rem_range = Totalrange               

                if indicator_canRefuel==2:
                    CoveredObjectiveValue += flows[j]
    
                
    # endtime = time.time()
    # finaltime = endtime-startime
    # print('B2: '+str(finaltime))
    return CoveredObjectiveValue


In [11]:
uniqueCS = []
for i in B.nodes:
    if B.nodes[i]['CS']==1:
        uniqueCS.append(i)
len(uniqueCS)

330

In [12]:
#Flow covered for different ranges with current situation
rng_200km = Objectivefunction(uniqueCS,df,Totalrange=170000)
rng_300km = Objectivefunction(uniqueCS,df,Totalrange=325000)
rng_600km = Objectivefunction(uniqueCS,df,Totalrange=585000)
print("Flow Covered for 200km",rng_200km, "300km",rng_300km, "600km",rng_600km)

Flow Covered for 200km 0.879328208606569 300km 0.9999999999999964 600km 0.9999999999999964
